In [1]:
import pickle
import numpy as np
import mne
from mne.preprocessing import ICA
import sys
sys.path.append('utils/')
from config import * #contains preprocessing hyperparameters

In [ ]:
FILE_PATH = './data/sub1.bdf' #path to the data
SAVE_PATH = './preprocessed/sub1.fif' #path to save preprocessed data

#Load and read data
raw = mne.io.read_raw_bdf(FILE_PATH, preload=True)
picks= mne.pick_types(raw.info, meg= False, eeg=True, eog=True, stim= True)

#Visual inspection of data to mark bad channels
order = np.arange(raw.info['nchan'])
raw.plot(n_channels=15, order=order, show=True)

In [ ]:
#Set EEG channel types and drop unnecessary channels
raw.drop_channels(DROPPED_CHANNELS)
raw.set_channel_types(mapping={'EXG1': 'eog','Status':'stim'})


In [ ]:
#Notch filter and Bandpass filter with zero phase
raw.notch_filter(np.arange(60, 241, 60), picks=picks, filter_length='auto',phase='zero')
raw.filter(LFREQ, HFREQ, picks=picks, filter_length='auto', phase='zero', fir_design='firwin')

In [ ]:
#Set EEG Montage
#This would typically be the digitized points but can also set average head
montage= mne.channels.read_montage('biosemi128')
raw.set_montage(montage)

In [ ]:
#Eye artifact removal using ICA Analysis of Data
n_components = 25 
method = 'fastica'
random_state = 15
ica = ICA(n_components=n_components, method= method, random_state= random_state)
ica.fit(raw, picks=picks)

# find which ICs match the EOG pattern
eog_indices, eog_scores = ica.find_bads_eog(raw)
ica.exclude = eog_indices

In [ ]:
# barplot of ICA component "EOG match" scores
ica.plot_scores(eog_scores)

# plot ICs applied to raw data, with EOG matches highlighted
ica.plot_sources(raw, show_scrollbars=True)

In [ ]:
ica.apply(raw)
raw.save(SAVE_PATH, picks=picks, overwrite=True)